# Process Kinder Morgan meter data

- Code Author: Sahar H El Abbadi
- Date Created: 2023-03-29
- Last Edited: 2023-03-29

Code to extract select Kinder Morgan pipeline data and calculated associated uncertainty


In [1]:
# Imports
import pandas as pd
import pathlib
import datetime

# Load data
km_df = pd.read_csv (pathlib.PurePath('02_meter_data', 'km_gas_comp_all_days.csv'), parse_dates=['datetime'])

# Load gas composition spreadsheet that contains refill dates and the su_raw and su_normalized values
import_cols = ['refill_no', 'rawhide_refill_date', 'start_utc', 'end_utc', 'notes', 'su_notes', 'km_composition_date', 'su_raw', 'su_normalized']
gas_comp_summary = pd.read_csv(pathlib.PurePath('02_meter_data', 'gas_comp_clean_su.csv'),
                               nrows=15, usecols=import_cols, parse_dates=['start_utc', 'end_utc'])

## KM gas composition: mean and standard deviation

#### Mean Calculations
KM provided data for two taps that fed into the filling location. As a conservative estimate and to account for latency in gas refill system (ie if gas is stored onsite for several days prior to use), we calculate a five-day average for mean gas composition. First, we calculate a daily average across the two taps. Next, we average the daily value across five days, inclusive of the refill date as the final day. Thus, for a gas refill on October 5th, we average gas composition from October 1st through October 5th.

#### Standard Deviation calculations
We calculate the standard deviation (1-sigma) using the same approach as for calculating the mean

In [2]:
# Calculate average gas composition across the two taps
km_df['tap_mean'] = km_df[['tap_1', 'tap_2']].mean(axis=1)

# Calculate the average gas composition over five days: the gas refill day, and the four days prior
km_gas_comp_mean = {}
for index, row in gas_comp_summary.iterrows():
    refill_no = row['refill_no']
    km_composition_date = row['km_composition_date']
    # km_composition_date is the end time for averaging
    end_t = datetime.datetime.strptime(km_composition_date, '%Y-%m-%d')
    delta_t = pd.Timedelta(days=4)
    start_t = end_t - delta_t

    # Select averaging period
    time_mask = (km_df['datetime'] >= start_t) & (km_df['datetime'] <= end_t)
    ave_period = km_df.loc[time_mask].copy()

    # Calculate mean and sigma values
    ave_period_mean = ave_period['tap_mean'].mean()
    ave_period_sigma = ave_period['tap_mean'].std()

    # Save in dictionary
    km_gas_comp_mean[refill_no] = [refill_no, ave_period_mean, ave_period_sigma]

# Merge the average gas composition with the gas_comp_clean_su_km
km_gas_comp_mean = pd.DataFrame.from_dict(km_gas_comp_mean, orient='index',
                                          columns=['refill_no', 'km', 'km_sigma'])
gas_comp_summary = gas_comp_summary.merge(km_gas_comp_mean, on='refill_no')

## SU gas composition: mean and standard deviation

The Stanford team collected one or two gas samples from each truck refill. Samples were collected using canisters provided by Eurofins AirToxics laboratory in Fulsom (CA), where laboratory compositional analysis was conducted. We excluded results where canisters arrived at the laboratory with vacuum level exceeding 15 inches of mercury. Where multiple samples were available, we take the average of the two. Where samples were missing, we take the average of the samples taken on immediately before and after the missing sample dates.

Eurofins AirToxics Laboratory used Modified ASTM D-1945 to analyze samples using a gas chromatograph with two detectors, a flame ionization detector (FID) and a thermal conductivity detector (TCD). Quantification was conducted on the following: oxygen, nitrogen, carbon monoxide, methane, carbon dioxide, ethane, ethene, acetylene, propane, isobutane, butane, neopentane, isopoentane, pentane.

In the report provided by Eurofins Air Toxins Laboratory, the sum of the individual constituents of the gas sample provided was greater than 100%. Additionally, several samples included values for percent methane which were 100%, which does not align with typical values observed for compressed natural gas. Furthermore, percent recovery for quality control standards run with experimental samples ranged from 95% to 101%, potentially indicative of variability associated with sample loss during measurement.


##### SU Raw:
Here, we use raw values reported by Eurofins Air Toxins laboratory for gas composition, with averages as described above.

##### Su Normalized:
Here, we normalize the gas compositional values such that the sum of all components is 100%.

## Uncertainty in Eurofins Analysis

Eurofins AirToxics Laboratory provided quality control data for all internal standards analyzed on the gas chromatographs used for analyzing Stanford's samples, over the time period most aligned with when Stanford sample analysis was being conducted.

Eurofins AirToxics uses samples of known composition as quality control standards, and measures the percent recovery. Samples at Stanford were analyzed from October 28th, 2022 through December 9th, 2022. Eurofins provided quality control data for Q4 of 2022, which includes all standards analyzed from November 21, 2022 through February 21, 2023. During this period, the 95% confidence interval for methane percent recovery was 92% to 101%, corresponding to a 2-sigma value of 4.5%.

Eurofins did have information to allow us to account for the systematic bias observed in measurements associated with percent recoveries of standards not equal to 100% .

Due to the high level of uncertainty associated with the Eurofins measurements, we use Kinder Morgan analysis as default values.

In [3]:
# Add uncertainty associated with su_raw and su_normalized values

two_sigma = 0.045
gas_comp_summary['su_raw_sigma'] = two_sigma/2
gas_comp_summary['su_normalized_sigma'] = two_sigma/2

In [4]:
# Save values

gas_comp_summary.to_csv(pathlib.PurePath('02_meter_data', 'gas_comp_clean_su_km.csv'))

In [5]:
from methods_source import select_methane_fraction

